In [ ]:
"""This is a simple code to convert the raw .jpeg thermal from djiM3T to .tif. This is a simple demonstration 
   of the use of dji_irp.exe under the latest version of the dji_thermal_sdk
   
   Thus, we are not using exiftool here to get and copy the metadata. Please download and use exiftool.exe for that. """

import subprocess
import os
import numpy as np
import tifffile
import matplotlib.pyplot as plt

# -------------------- INPUT/OUTPUT --------------------
input_file = r"path to the input file in .jpeg"
output_dir = r"Output directory path here"
os.makedirs(output_dir, exist_ok=True)

# DJI TSDK Executable
dji_irp = r"...........\utility\bin\windows\release_x64\dji_irp.exe" #the path to the dji.irp.exe in the latest version of dji_thermal_sdk

# Environmental correction parameters
emissivity = 0.95
atmp = 25            # Ambient temperature in °C
humidity = 80        # Relative humidity in %
distance = 15        # Drone height in meters
refl_temp = 23       # Reflected apparent temperature in °C

# DJI M3T RAW dimensions
raw_width = 640
raw_height = 512

# -------------------- PROCESS IMAGE --------------------
basename = os.path.splitext(os.path.basename(input_file))[0]

# Measure Temperature (float32) with corrections
measure_file = os.path.join(output_dir, f"{basename}_temperature.raw")
subprocess.run([
    dji_irp,
    "-s", input_file,
    "-a", "measure",
    "-o", measure_file,
    "--measurefmt", "float32",
    "--emissivity", str(emissivity),
    "--ambient", str(atmp),
    "--humidity", str(humidity),
    "--distance", str(distance),
    "--reflection", str(refl_temp)
], check=True)

# Convert RAW → float32 TIFF
tif_file = os.path.join(output_dir, f"{basename}_temperature.tif")
data = np.fromfile(measure_file, dtype=np.float32).reshape((raw_height, raw_width))
tifffile.imwrite(tif_file, data)

# Display temperature map
plt.figure(figsize=(8,6))
plt.imshow(data, cmap='inferno')
plt.colorbar(label='Temperature (°C)')
plt.title(f"Thermal Image - {basename}")
plt.show()

# Print pixel statistics
print(f"Min: {np.min(data):.2f} °C")
print(f"Max: {np.max(data):.2f} °C")
print(f"Mean: {np.mean(data):.2f} °C")
print(f" TIFF saved at: {tif_file}")
